# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [34]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/origenolet/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/origenolet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/origenolet/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
data = pd.read_csv("./training.1600000.processed.noemoticon.csv", 
                  encoding='latin-1', header = None)
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
data.columns = ['target','id','date','flag','user','text']
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
data = data.sample(n = 20000, random_state = 50, replace = False)
print(data.shape)
data.head()

(20000, 6)


,target,id,date,flag,user,text
1579740,4,2189983665,Tue Jun 16 01:26:25 PDT 2009,NO_QUERY,SarahMorrison,I love you LA. Its nice to be home
504647,0,2188186225,Mon Jun 15 21:26:57 PDT 2009,NO_QUERY,jjbabyMEOWZER,She back and i really want to see her but i do...
815879,4,1551270859,Sat Apr 18 08:36:26 PDT 2009,NO_QUERY,BigcityBrooke3,Tonight is going to be Fucking amazing &lt;3333
199512,0,1971455566,Sat May 30 07:09:13 PDT 2009,NO_QUERY,lilwldchld,@aholston aww... I hope you get everything do...
1238232,4,1993226101,Mon Jun 01 10:02:06 PDT 2009,NO_QUERY,SarrisW,"@aaronklein @mchesner Being an artist type, ..."


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
def clean_up(s):
    s = s.lower()
    s = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ' , s)
    s = re.sub(r'\W+',' ',s) 
    s = re.sub(r'\s+',' ',s) 
    s = re.sub(r'\d+',' ',s) 
    s = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)'," ", s)  
    s = BeautifulSoup(s, 'lxml').get_text().strip()
    return s

In [6]:
def tokenize(s):
    s = word_tokenize(s)
    return s

In [7]:
def stem_and_lemmatize(l):
    lst = []
    lem = WordNetLemmatizer()
    for item in l:
        item = lem.lemmatize(item)
        lst.append(item)
    return lst

In [8]:
def remove_stopwords(l):   
    stop_words = list(stopwords.words('english')) 
    for item in l:
        if item in stop_words:
            l.remove(item)
        else:
            pass
    return l

In [9]:
functions = [clean_up, tokenize, stem_and_lemmatize, remove_stopwords]
cols = ["text", "text_processed", "text_processed", "text_processed"]
for function, col in zip(functions, cols):
    data["text_processed"] = data[col].apply(function)
    
data.head()

,target,id,date,flag,user,text,text_processed
1579740,4,2189983665,Tue Jun 16 01:26:25 PDT 2009,NO_QUERY,SarahMorrison,I love you LA. Its nice to be home,"[love, la, nice, be, home]"
504647,0,2188186225,Mon Jun 15 21:26:57 PDT 2009,NO_QUERY,jjbabyMEOWZER,She back and i really want to see her but i do...,"[back, really, want, see, but, i, dont, enough..."
815879,4,1551270859,Sat Apr 18 08:36:26 PDT 2009,NO_QUERY,BigcityBrooke3,Tonight is going to be Fucking amazing &lt;3333,"[tonight, going, be, fucking, amazing, lt]"
199512,0,1971455566,Sat May 30 07:09:13 PDT 2009,NO_QUERY,lilwldchld,@aholston aww... I hope you get everything do...,"[aholston, aww, hope, get, everything, done]"
1238232,4,1993226101,Mon Jun 01 10:02:06 PDT 2009,NO_QUERY,SarrisW,"@aaronklein @mchesner Being an artist type, ...","[aaronklein, mchesner, an, artist, type, t, re..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [10]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [11]:
flat_list = flatten(data["text_processed"])
flat_list

['love',
 'la',
 'nice',
 'be',
 'home',
 'back',
 'really',
 'want',
 'see',
 'but',
 'i',
 'dont',
 'enough',
 'courage',
 'im',
 'scared',
 'do',
 'alone',
 'tonight',
 'going',
 'be',
 'fucking',
 'amazing',
 'lt',
 'aholston',
 'aww',
 'hope',
 'get',
 'everything',
 'done',
 'aaronklein',
 'mchesner',
 'an',
 'artist',
 'type',
 't',
 'resist',
 'jab',
 'good',
 'morning',
 'everyone',
 'ready',
 'start',
 'another',
 'exciting',
 'week',
 'degree',
 'outside',
 'net',
 'at',
 'hme',
 'rbflygal',
 's',
 'bridal',
 'shower',
 'without',
 'sex',
 'toy',
 'going',
 'do',
 'uni',
 'work',
 'wish',
 'luck',
 'thisisdavina',
 'awesome',
 'ack',
 'sure',
 'mum',
 'bought',
 'workout',
 'i',
 'swear',
 'never',
 'stop',
 'complaining',
 'still',
 'doe',
 'rosyantoine',
 'doe',
 'do',
 'shayboo',
 'lyfe',
 'yeah',
 'm',
 'going',
 'be',
 'meeting',
 'on',
 'july',
 'th',
 'wonder',
 'whether',
 'fukuoka',
 'knew',
 'iata',
 'code',
 'wa',
 'going',
 'be',
 'fuk',
 'two',
 'day',
 'till',


In [12]:
fdist = FreqDist(flat_list)
fdist

FreqDist({'the': 3329, 'i': 2454, 'a': 2083, 'my': 1476, 'day': 1364, 'm': 1292, 'wa': 1265, 't': 1221, 's': 1184, 'get': 1142, ...})

In [13]:
new_dist = {}
for key, value in fdist.items():
    if key == 'the':
        pass
    elif len(key) > 2:
        new_dist[key]= value
    else:
        pass

In [14]:
new_dist

{'love': 838,
 'nice': 294,
 'home': 477,
 'back': 730,
 'really': 617,
 'want': 632,
 'see': 538,
 'but': 209,
 'dont': 219,
 'enough': 84,
 'courage': 3,
 'scared': 37,
 'alone': 56,
 'tonight': 320,
 'going': 818,
 'fucking': 46,
 'amazing': 154,
 'aholston': 1,
 'aww': 117,
 'hope': 445,
 'get': 1142,
 'everything': 108,
 'done': 187,
 'aaronklein': 1,
 'mchesner': 1,
 'artist': 10,
 'type': 22,
 'resist': 4,
 'jab': 3,
 'good': 1129,
 'morning': 496,
 'everyone': 197,
 'ready': 172,
 'start': 167,
 'another': 185,
 'exciting': 16,
 'week': 338,
 'degree': 18,
 'outside': 83,
 'net': 18,
 'hme': 1,
 'rbflygal': 1,
 'bridal': 1,
 'shower': 65,
 'without': 113,
 'sex': 27,
 'toy': 15,
 'uni': 19,
 'work': 860,
 'wish': 419,
 'luck': 116,
 'thisisdavina': 5,
 'awesome': 230,
 'ack': 2,
 'sure': 181,
 'mum': 46,
 'bought': 50,
 'workout': 27,
 'swear': 12,
 'never': 221,
 'stop': 97,
 'complaining': 4,
 'still': 531,
 'doe': 124,
 'rosyantoine': 1,
 'shayboo': 1,
 'lyfe': 1,
 'yeah': 2

In [15]:
new_dist = {k: v for k, v in sorted(new_dist.items(), key=lambda item: item[1], reverse=True)}
new_dist

{'day': 1364,
 'get': 1142,
 'good': 1129,
 'you': 1107,
 'like': 957,
 'quot': 893,
 'have': 876,
 'got': 872,
 'work': 860,
 'love': 838,
 'time': 829,
 'today': 825,
 'going': 818,
 'one': 743,
 'lol': 737,
 'back': 730,
 'know': 667,
 'want': 632,
 'really': 617,
 'amp': 601,
 'well': 582,
 'night': 561,
 'can': 560,
 'need': 556,
 'think': 551,
 'see': 538,
 'still': 531,
 'new': 518,
 'morning': 496,
 'thanks': 490,
 'home': 477,
 'that': 471,
 'feel': 470,
 'miss': 465,
 'just': 459,
 'twitter': 448,
 'much': 447,
 'hope': 445,
 'great': 443,
 'not': 442,
 'make': 438,
 'tomorrow': 438,
 'last': 433,
 'wish': 419,
 'are': 418,
 'all': 410,
 'and': 380,
 'haha': 374,
 'sad': 363,
 'bad': 349,
 'fun': 346,
 'sorry': 342,
 'would': 339,
 'week': 338,
 'come': 334,
 'happy': 328,
 'sleep': 328,
 'thing': 325,
 'tonight': 320,
 'this': 318,
 'way': 310,
 'friend': 309,
 'though': 308,
 'right': 306,
 'will': 305,
 'don': 305,
 'for': 303,
 'hate': 303,
 'gon': 301,
 'had': 295,
 'nic

In [16]:
corpus = list(new_dist.keys())[:5000]

In [17]:
len(corpus)

5000

In [18]:
corpus

['day',
 'get',
 'good',
 'you',
 'like',
 'quot',
 'have',
 'got',
 'work',
 'love',
 'time',
 'today',
 'going',
 'one',
 'lol',
 'back',
 'know',
 'want',
 'really',
 'amp',
 'well',
 'night',
 'can',
 'need',
 'think',
 'see',
 'still',
 'new',
 'morning',
 'thanks',
 'home',
 'that',
 'feel',
 'miss',
 'just',
 'twitter',
 'much',
 'hope',
 'great',
 'not',
 'make',
 'tomorrow',
 'last',
 'wish',
 'are',
 'all',
 'and',
 'haha',
 'sad',
 'bad',
 'fun',
 'sorry',
 'would',
 'week',
 'come',
 'happy',
 'sleep',
 'thing',
 'tonight',
 'this',
 'way',
 'friend',
 'though',
 'right',
 'will',
 'don',
 'for',
 'hate',
 'gon',
 'had',
 'nice',
 'getting',
 'better',
 'watching',
 'look',
 'say',
 'yeah',
 'your',
 'hour',
 'weekend',
 'wait',
 'could',
 'take',
 'bed',
 'yes',
 'too',
 'guy',
 'next',
 'people',
 'school',
 'even',
 'best',
 'feeling',
 'awesome',
 'thank',
 'now',
 'hey',
 'never',
 'dont',
 'working',
 'year',
 'wan',
 'with',
 'cant',
 'tweet',
 'but',
 'let',
 'there

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [19]:
def find_features(document):
    words = set(document)
    features = {}
    for w in corpus:
        features[w] = (w in words)

    return features

In [20]:
features = []
for i in range(0,len(data["text_processed"])):
    x = find_features(list(data["text_processed"])[i])
    features.append(x)

In [24]:
is_positive = []
analyzer = SentimentIntensityAnalyzer()
for i in range(0,len(data["text"])):
    x = analyzer.polarity_scores(list(data["text"])[i])
    if x["compound"] > 0:
        is_positive.append(True)
    elif x["compound"] == 0:
        is_positive.append(False)
    else:
        is_positive.append(False)

In [25]:
is_positive[:10]

[True, False, True, True, False, True, False, False, True, True]

In [29]:
print(is_positive.count(True))
print(is_positive.count(False))

9342
10658


In [ ]:
### There is no need to correct class imbalance, since the number of positive reviews 
### is more or less equal to the number of negative reviews

In [26]:
data_model = list(zip(features, is_positive))

In [30]:
len(data_model)

20000

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [32]:
training_set = data_model[:16000]
testing_set = data_model[16000:]

In [35]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [36]:
classifier.show_most_informative_features(20)

Most Informative Features
               beautiful = True             True : False  =     39.0 : 1.0
                  thanks = True             True : False  =     29.5 : 1.0
                  loving = True             True : False  =     27.8 : 1.0
                 awesome = True             True : False  =     23.8 : 1.0
                    xoxo = True             True : False  =     23.2 : 1.0
                enjoying = True             True : False  =     22.6 : 1.0
                   thank = True             True : False  =     21.9 : 1.0
                     thx = True             True : False  =     21.7 : 1.0
                congrats = True             True : False  =     20.9 : 1.0
                 welcome = True             True : False  =     20.4 : 1.0
                    love = True             True : False  =     17.6 : 1.0
                   smile = True             True : False  =     17.6 : 1.0
                   worst = True            False : True   =     17.2 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [38]:
print("Classifier accuracy score:",(nltk.classify.accuracy(classifier, testing_set)))

Classifier accuracy score: 0.8245


In [ ]:
### Yey! Accuracy of 0.8245 :-)

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here